# Using SQL to Answer Business Questions

Goal: To demonstrate the use of sql (mysql) to query database to answer business questions.
The database in question is from a company involved in 'Food and Beverage Industry'.

<font color='Green'>Note</font>:
- The questions and database are part of the *8weeksqlchallenge*. Get to know more on this [Link](https://8weeksqlchallenge.com/case-study-1/)
- To connect to the database, replace " config['host'],config['password']['lrng'], config['user'][0],config['database'][0] " with the name of your mysql server, password, user, and database respectively. These parameters are supplied in this project from a configuration file. Read more [Link](https://towardsdatascience.com/from-novice-to-expert-how-to-write-a-configuration-file-in-python-273e171a8eb3#:~:text=A%20good%20configuration%20file%20should,be%20only%20read%20by%20developers.)

In [32]:
import mysql.connector
from access import config   # using python config.py method to input access codes and protect password
conn_ = mysql.connector.connect(
    host = config['host'],
    password = config['password']['lrng'], 
    user= config['user'][0],
    database = config['database'][0])
print(conn_)

## Checking the Tables In the Database

- To check the tables in the database, use :
    - connect.cursor() method to instatiate a cursor instance through which you access the database
    - cursor.execute() method to execute the sql query
    - use any of cursor.fetchall(), cursor.fetchone() or cursor.fetchmany() to access the output of the query

In [33]:


cur_ = conn_.cursor()
cur_.execute("show tables")

myresult = cur_.fetchall()      # fetchall() is used to fetch all output from the query
print(myresult)

[('members',), ('menu',), ('sales',)]


In [34]:
cur_.execute("desc members")
members_desc = cur_.fetchall()
members_desc

[('customer_id', b'varchar(1)', 'YES', '', None, ''),
 ('join_date', b'date', 'YES', '', None, '')]

In [35]:
cur_.execute("desc menu")
menu_desc = cur_.fetchall()
menu_desc

[('product_id', b'int', 'YES', '', None, ''),
 ('product_name', b'varchar(5)', 'YES', '', None, ''),
 ('price', b'int', 'YES', '', None, '')]

In [36]:
cur_.execute("desc sales")
sales_desc = cur_.fetchall()
sales_desc

[('customer_id', b'varchar(1)', 'YES', '', None, ''),
 ('order_date', b'date', 'YES', '', None, ''),
 ('product_id', b'int', 'YES', '', None, '')]

# Answering Business Questions

### 1. What is the total amount each customer spent at the restaurant?

In [39]:
query_ = ''' 
SELECT s.customer_id, SUM(m.price) total_spent_cust
FROM sales s
LEFT JOIN menu m
ON s.product_id = m.product_id
GROUP BY s.customer_id
ORDER BY sum(m.price) asc
'''

cur_.execute(query_)
total_spent_per_customer = cur_.fetchall()
for x in total_spent_per_customer:
    print(x)

('C', Decimal('36'))
('B', Decimal('74'))
('A', Decimal('76'))


### 2. How many days has each customer visited the restaurant?

In [43]:
query_ = '''
SELECT customer_id, COUNT( DISTINCT order_date) days_cust_visit
FROM sales
GROUP BY 1
'''
cur_.execute(query_)
total_days_visit_per_cust = cur_.fetchall()
for x in total_days_visit_per_cust:
    print(x)

('A', 4)
('B', 6)
('C', 2)


### 3. What was the first item from the menu purchased by each customer?

To answer this question, we are going to use <font color ='red'>**correlated subqueries** </font>.

In [45]:
query_ = '''
SELECT s.customer_id, m.product_name
FROM sales s
LEFT join menu m
ON s.product_id = m.product_id
WHERE s.order_date = ( 
    SELECT min(s1.order_date) 
    FROM sales s1
    WHERE s.customer_id = s1.customer_id)
'''

cur_.execute(query_)

fim_purchase_per_cust = cur_.fetchall()
fim_purchase_per_cust

[('A', 'sushi'),
 ('A', 'curry'),
 ('B', 'curry'),
 ('C', 'ramen'),
 ('C', 'ramen')]

### 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [48]:
query_ = '''
SELECT s.customer_id, m.product_name, COUNT(s.customer_id)
FROM sales s
LEFT JOIN menu m
ON s.product_id = m.product_id
WHERE s.product_id = 
( 
    SELECT product_id
    FROM
        ( 
            SELECT product_id, COUNT(product_id) counts
            FROM sales
            GROUP BY product_id
        ) s1
        WHERE counts = 
        (
            SELECT MAX(counts)
            FROM 
            (
                SELECT product_id, COUNT(product_id) counts 
                FROM sales
                GROUP BY product_id
            ) s2
        )
)
GROUP BY 1,2
'''
# s2 subquery is used to generate the product_id with the maximum counts
# s1 subquery is used to isolate the aggregated counts for each product_id which is then compared to the max(counts) to 
# isolate the product_id with the max_counts

cur_.execute(query_)
mpi_menu_purchase_per_cust = cur_.fetchall()
for x in mpi_menu_purchase_per_cust:
    print(x)

('A', 'ramen', 3)
('B', 'ramen', 2)
('C', 'ramen', 3)


### 5. Which item was the most popular for each customer?

using uncorrelated subqueries to answer business questions

In [49]:
query_ = '''
SELECT s.customer_id, m.product_name, COUNT(s.customer_id)
FROM sales s
LEFT JOIN menu m
ON s.product_id = m.product_id
WHERE m.product_name = ANY 
(
    SELECT product_name
    FROM 
    (
        SELECT  m.product_name, COUNT(m.product_name) counts
        FROM sales s1
        LEFT JOIN menu m
        ON s1.product_id = m.product_id
        WHERE s.customer_id = s1.customer_id
        GROUP BY 1
    ) prdt1
    WHERE counts = 
    (
        SELECT max(counts)
        FROM 
        (
            SELECT m.product_name, COUNT(m.product_name) counts
            FROM sales s1
            LEFT JOIN menu m
            ON s1.product_id = m.product_id
            WHERE s.customer_id = s1.customer_id
            GROUP BY 1
        ) prdt1

    ) 
)
GROUP BY 1,2
'''

# 'prdt1' is an uncorrelated subquery used to compute the product_name with the max counts for each customer

cur_.execute(query_)

mpi_each_cust = cur_.fetchall()
for x in mpi_each_cust:
    print(x)

('A', 'ramen', 3)
('B', 'curry', 2)
('B', 'sushi', 2)
('B', 'ramen', 2)
('C', 'ramen', 3)


###  6. Which item was purchased first by the customer after they became a member?

Using the rank() function with the window function (mysql) to answer business question:
- we use rank() to number the purchase by each customer based on the date the items were procured

In [50]:
query_ = '''
SELECT *
FROM
(
    SELECT s.customer_id, m.product_name , m_.join_date, s.order_date,
    rank() over (partition by s.customer_id order by s.order_date) as position
    FROM sales s
    LEFT JOIN menu m
    ON s.product_id = m.product_id
    LEFT JOIN members m_
    ON s.customer_id = m_.customer_id
    WHERE s.order_date >= m_.join_date
) customer_join_first_purchase
WHERE position = 1
'''

cur_.execute(query_)

ip_cust_after_member = cur_.fetchall()

ip_cust_after_member

[('A', 'curry', datetime.date(2021, 1, 7), datetime.date(2021, 1, 7), 1),
 ('B', 'sushi', datetime.date(2021, 1, 9), datetime.date(2021, 1, 11), 1)]

### 7. Which item was purchased just before the customer became a member?

In [51]:


query_ = '''
SELECT *
FROM 
(
    SELECT s.customer_id, m.product_name , m_.join_date, s.order_date,
        rank() over (partition by s.customer_id order by s.order_date desc) as position
    FROM sales s
    LEFT JOIN menu m
    ON s.product_id = m.product_id
    LEFT JOIN members m_
    ON s.customer_id = m_.customer_id
    WHERE s.order_date < m_.join_date
) customer_join_first_purchase
where position = 1
'''

cur_.execute(query_)

ip_cust_before_member = cur_.fetchall()
ip_cust_before_member

[('A', 'sushi', datetime.date(2021, 1, 7), datetime.date(2021, 1, 1), 1),
 ('A', 'curry', datetime.date(2021, 1, 7), datetime.date(2021, 1, 1), 1),
 ('B', 'sushi', datetime.date(2021, 1, 9), datetime.date(2021, 1, 4), 1)]

### 8. What is the total items and amount spent for each member before they became a member?

You can also use "USING" keyword insted of "ON" keyword when join columns with identical names.
 
Note: that the common column name must be in parenthesis. "USING" keyword works only in mysql

In [53]:
query_ = '''
SELECT s.customer_id, count(s.product_id) total_item, sum(m.price) amount_spent
FROM sales s
LEFT JOIN menu m
USING (product_id)
LEFT JOIN members m_
USING (customer_id)
WHERE s.order_date < m_.join_date
GROUP BY 1
'''

cur_.execute(query_)

total_items_amount_per_cust_before_member = cur_.fetchall()
for x in total_items_amount_per_cust_before_member:
    print(x)

('A', 2, Decimal('25'))
('B', 3, Decimal('40'))


### 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

- Using conditional statement "<font color='red'>**CASE**</font>", sets, and subqueries to answer business questions.
    - To answer the question above, we will use:
        1. "CASE" clause to create 2 (two) columns based on join_date/order_date and sushi/other items
        2. "UNION" clause to join records matching the prerequiste criteria

In [55]:
query_ = '''
SELECT customer_id, sum(points) total_points, MONTHNAME(order_date) month_
FROM 
(
    SELECT * 
    FROM 
    (
        SELECT s.customer_id, m.price, m.product_name, s.order_date, m_.join_date, 
        CASE 
        WHEN s.order_date BETWEEN m_.join_date AND DATE_ADD(m_.join_date, interval 6 day) THEN 1
        ELSE 0
        END AS cond_date ,
        CASE
        WHEN s.order_date BETWEEN m_.join_date AND DATE_ADD(m_.join_date, interval 6 day) THEN m.price *10*2
        ELSE 0
        END AS points
        FROM sales s
        LEFT JOIN menu m ON s.product_id = m.product_id
        LEFT JOIN members m_ ON s.customer_id = m_.customer_id
    ) date_cond
    WHERE cond_date = 1
    UNION
    SELECT *
    FROM 
    (
        SELECT s.customer_id,m.price, m.product_name, s.order_date, m_.join_date,
        CASE
        WHEN s.order_date < m_.join_date OR s.order_date > DATE_ADD(m_.join_date, interval 6 day) THEN 2
        ELSE 0
        END AS cond_date ,
        CASE
        WHEN m.product_name = 'sushi' THEN m.price * 10 * 2
        ELSE m.price * 10
        END AS points
        FROM sales s
        LEFT JOIN menu m ON s.product_id = m.product_id
        LEFT JOIN members m_ ON s.customer_id = m_.customer_id
    ) date_cond
    WHERE cond_date = 2
) first_week_sushi_filter
WHERE MONTHNAME(order_date) = 'January'
GROUP BY 1,3
'''

cur_.execute(query_)

points_based_week_join_sushi = cur_.fetchall()
for x in points_based_week_join_sushi:
    print(x)

('A', Decimal('1130'), 'January')
('B', Decimal('820'), 'January')


Based on the result of our query, customer 'A' has 1130 points and customer 'B' has 820 points